# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Cell 1: Clean setup
import os
import urllib3
import requests

# Disable SSL warnings (note the capital W in Warning)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Simple approach: Patch at the Session level
original_request = requests.Session.request

def no_ssl_request(self, method, url, **kwargs):
    kwargs.setdefault('verify', False)
    return original_request(self, method, url, **kwargs)

requests.Session.request = no_ssl_request

In [2]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [3]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    os.environ["LANGSMITH_ENDPOINT"] = "https://eu.api.smith.langchain.com"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 8f728f46


In [5]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent(model='gpt-5-nano')
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [6]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [9]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [10]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [7]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: c:\Users\dznidaric\Documents\github\AIE9\07_Deep_Agents\workspace


In [8]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[DIR] research


In [9]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (252 bytes)


In [10]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [13]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("gpt-5-nano"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: c:\Users\dznidaric\Documents\github\AIE9\07_Deep_Agents\workspace


In [14]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Thanks for the details. I’ve created a personalized sleep improvement plan and saved it to a file. Here’s what I did and what’s inside.

What I did
- Created a set of 10 structured todos to guide your sleep plan (baseline tracking, wind-down routine, environment, substances, morning routine, diary, review, etc.).
- Saved your plan content to /sleep_improvement_plan.txt for easy reference and weekly revisit.
- Marked the final planning step as completed.

Where to find the plan
- File: /sleep_improvement_plan.txt
- It contains the full plan, including rationale, goals, and step-by-step actions you can follow.

Key elements of your personalized plan (highlights)
- Sleep window and routine
  - Bedtime: 11:00 pm (with flexibility of ±30 minutes if needed)
  - Wake time: 7:00 am
  - Wind-down: Starts around 10:00–10:30 pm, 30–60 minutes total
    - Dim lights, no screens, relaxing activities (reading, light stretching, journaling)
    - 5 minutes of slow, diaphragmatic or bo

In [15]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
⬜ [todo_1] Baseline sleep tracking (5 nights) (pending)
⬜ [todo_3] Set target sleep window (pending)
⬜ [todo_5] Design a 30–60 minute wind-down routine (pending)
⬜ [todo_7] Eliminate phone use in bed (pending)
⬜ [todo_9] Optimize sleep environment (pending)
⬜ [todo_11] Caffeine and meals timing (pending)
⬜ [todo_13] Establish a consistent morning routine (pending)
⬜ [todo_15] Create a simple sleep diary template (pending)
⬜ [todo_17] 14-day review and adjustment plan (pending)
✅ [todo_19] Save and review plan file (completed)


Workspace contents:
  [DIR] research/
  [FILE] sleep_improvement_plan.txt (3470 bytes)


---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
The value depends on task structure and uncertainty. Planing is unnecessary cost when task is of low complexity, that has more planning steps than execution steps. Also situation where we are doing heavy exploration (research, debugging, creative ideation), we often don’t know the next step until after acting.
Based on the best algorithms for optimizationm todo items should divide and conquer. Usually we should follow these sayings: "If you can’t clearly say what “done” means, it’s too coarse, and if completing it feels trivial or bureaucratic, it’s too fine."
Completion, revision, or deletion should be enforced so that the agent is sure to complete all the task, or get rid of the tasks that are obsolete.

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
I would store large health document and user condition in files, based on the agents goal, use rag to get the data or more information. The things that should stay in the prompt is current goal, only today's users metric summary and short list of user conditions for safety.
Large scale would be on files, intent in the prompt and user safety in both. 

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [21]:
from pathlib import Path

workspace_path = Path("workspace").absolute()
workspace_path.mkdir(exist_ok=True)


guide_src = Path("data/HealthWellnessGuide.txt").absolute()
guide_dst = workspace_path / "HealthWellnessGuide.txt"
guide_dst.write_text(guide_src.read_text(), encoding="utf-8")

16206

In [22]:
@tool
def retrieve_guide_section(query: str) -> str:
    """
    Return only relevant guide paragraphs.
    Very simple keyword-based retrieval.
    """
    text = guide_dst.read_text(encoding="utf-8")

    hits = []
    for para in text.split("\n\n"):
        if query.lower() in para.lower():
            hits.append(para.strip())

    return "\n\n".join(hits[:3]) or "No relevant section found."

In [24]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    retrieve_guide_section
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("gpt-5-nano"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print("✅ Research Agent created!")
print(f"📁 Workspace: {workspace_path}")


✅ Research Agent created!
📁 Workspace: c:\Users\dznidaric\Documents\github\AIE9\07_Deep_Agents\workspace


In [25]:
print("\n" + "="*60)
print("🔬 TESTING: Stress Management Research")
print("="*60 + "\n")

result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."""
    }]
})

print("\n📊 Agent Response:")
print(result["messages"][-1].content)

print("\n" + "="*60)
print("✅ Research Complete!")
print("="*60)
print(f"\n📋 Final Todo Status:")
print(list_todos.invoke({}))


🔬 TESTING: Stress Management Research


📊 Agent Response:
Here’s a concise, delivered-result summary of the stress management research guide I created, plus options for next steps.

Status and what’s saved
- Completed tasks: audience definition, structure outline, extraction and synthesis of evidence for multiple strategies, quick-start content, and drafting a comprehensive guide.
- Main guide file saved: /docs/stress_management_guide.md
  - Content includes 6 evidence-based strategies with practical steps, quick-start elements, and caveats.
  - Also includes a printable quick-start checklist section (lines 86–93 in the file).
- Next possible save: a dedicated quick-start printable file (/docs/quick_start_stress_guide.md) if you’d like a separate one-page version. I can create this now if desired.

What the guide covers (at-a-glance)
Audience: General adults seeking practical, evidence-informed stress management strategies.

Strategy 1. Mindfulness-Based Stress Reduction (MBSR) / Mind

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [33]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-5-nano",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [34]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("openai:gpt-4o"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [35]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
I've successfully created a comprehensive morning routine guide and saved it as a markdown file named `morning_routine_findings.md`. The guide includes:

1. **Research on Morning Routines**: A summary of the scientific findings highlighting the impact of morning routines on energy levels.
2. **Exercise Routines**: Practical, easy-to-follow exercise steps designed to boost morning energy.
3. **Nutritional Strategies**: Foods and dietary tips to enhance energy levels throughout the morning.
4. **Mindset Practices**: Mindfulness, affirmations, and other mindset techniques to kickstart the day positively.

You can download and view the markdown file for detailed information. If you need any modifications or a specific format, feel free to ask!


In [36]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Develop practical steps for morning exercise (completed)
✅ [todo_5] Include nutritional advice for morning routines (completed)
✅ [todo_7] Create mindset practices fitting for morning routines (completed)
✅ [todo_9] Compile research into a markdown file (completed)

Generated files in workspace:
  [DIR] docs/
  [FILE] HealthWellnessGuide.txt (16624 bytes)
  [FILE] HealthWellnessGuide_Summary.txt (1525 bytes)
  [DIR] research/
  [FILE] sleep_improvement_plan.txt (3470 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [37]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [38]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [39]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [ ]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [40]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [41]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [42]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [43]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("openai:gpt-5"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [ ]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [ ]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [44]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [45]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("openai:gpt-5"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [46]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Thanks, Alex! I’ve created your personalized 2-week wellness program and saved each component as a separate file for easy reference.

Files saved:
- Exercise plan: /user_alex_exercise_plan_2w.txt
- Nutrition plan: /user_alex_nutrition_plan_2w.txt
- Stress & sleep plan: /user_alex_stress_sleep_plan_2w.txt

What’s inside
- Exercise (3x/week, 30 mins, mornings): Two alternating full-body workouts with warm-up, main sets, cool-down, and breathwork. Bodyweight-first with optional dumbbell/band swaps, gentle week-2 progression, plus one 8–10 min mobility session each week. Includes low-energy/anxiety modifications, habit tips, and a simple tracking template.
- Nutrition (vegetarian): 2-week meal plan with 3 meals + 1–2 snacks/day, portion guidance, recipes, batch-prep guide, weekly grocery lists, pre/post-workout options, hydration and caffeine timing, and key vegetarian nutrients (iron, B12, D, calcium, iodine, zinc, omega-3) with food sources and supplement notes.


In [ ]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

In [ ]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
*Your answer here*

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
*Your answer here*

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [47]:
# Step 1: Define your subagent configurations
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [48]:
# Step 2: Create any additional tools you need
import json
from datetime import datetime, timedelta

# Daily Check-in Tool
CHECKIN_LOG = {}
@tool
def log_daily_checkin(user_id: str, day: int, completed_tasks: str, energy_level: int, notes: str) -> str:
    """Log a user's daily check-in for their wellness challenge.
    
    Args:
        user_id: The unique identifier for the user
        day: Day number (1-30)
        completed_tasks: Comma-separated list of completed tasks
        energy_level: Energy level 1-10
        notes: Any additional notes or feedback
    
    Returns:
        Confirmation with encouragement
    """
    if user_id not in CHECKIN_LOG:
        CHECKIN_LOG[user_id] = {}
    
    CHECKIN_LOG[user_id][f"day_{day}"] = {
        "day": day,
        "completed_tasks": completed_tasks,
        "energy_level": energy_level,
        "notes": notes,
        "timestamp": datetime.now().isoformat()
    }
    
    streak = len(CHECKIN_LOG[user_id])
    return f"✅ Day {day} logged! You're on a {streak}-day streak! Energy: {energy_level}/10"

@tool
def get_checkin_history(user_id: str, last_n_days: int = 7) -> str:
    """Retrieve recent check-in history for a user.
    
    Args:
        user_id: The unique identifier for the user
        last_n_days: Number of recent days to retrieve (default 7)
    
    Returns:
        JSON string with check-in history
    """
    if user_id not in CHECKIN_LOG:
        return f"No check-in history for {user_id}"
    
    history = dict(list(CHECKIN_LOG[user_id].items())[-last_n_days:])
    return json.dumps(history, indent=2)

@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    - habit-formation: Create personalized habits to follow.
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [50]:
# Step 3: Build the main coordinator agent
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("openai:gpt-5"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Logging
        log_daily_checkin,
        get_checkin_history,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach specializing in 30-Day Wellness Challenges.

## Your Role
Create personalized, sustainable 30-day wellness programs that integrate:
- Exercise and physical fitness
- Nutrition and healthy eating
- Stress management and mindfulness
- Sleep optimization

## Workflow for Creating a 30-Day Challenge

1. **User Assessment**
   - Get user profile (or create if new)
   - Understand current habits, goals, and constraints
   - Save ALL preferences (dietary restrictions, fitness level, time availability, goals)

2. **Planning Phase**
   - Create comprehensive todo list for program creation
   - Break down into: exercise plan, nutrition plan, mindfulness plan, integration

3. **Specialist Delegation**
   - **exercise-specialist**: Create progressive 30-day workout plan
   - **nutrition-specialist**: Create meal planning guidelines and weekly templates
   - **mindfulness-specialist**: Create stress management and sleep optimization routines

4. **Program Integration**
   - Combine specialist outputs into ONE cohesive 30-day plan
   - Organize by weeks (Week 1-4) with daily breakdown
   - Include rest/recovery days
   - Create realistic daily time commitment

5. **Documentation** (Save separate files)
   - `{user_id}_30day_challenge.md`: Complete integrated 30-day plan
   - `{user_id}_exercise_plan.md`: Detailed workout schedules
   - `{user_id}_nutrition_guide.md`: Meal planning and recipes
   - `{user_id}_mindfulness_routine.md`: Daily mindfulness practices
   - `{user_id}_weekly_checklist.md`: Simple weekly tracking checklist

6. **Weekly Summaries**
   - After weekly check-ins, create summary reports
   - Identify patterns in energy, completion, obstacles
   - Adapt recommendations for next week
   - Save as `{user_id}_week{N}_summary.md`

## Daily Check-ins
When user logs daily progress:
- Acknowledge their effort enthusiastically
- Log check-in with log_daily_checkin
- Review recent history if needed
- Provide encouragement and adaptive tips
- If patterns emerge (low energy, missed workouts), suggest modifications

## Adaptation Strategy
- Week 1: Focus on habit formation (load habit_formation skill)
- Week 2-3: Gradual intensity increase
- Week 4: Consolidation and planning for continuation
- Always: Safety first (load injury_prevention skill if needed)

## Important
- Be encouraging and supportive, not judgmental
- Respect limitations and constraints
- Make plans REALISTIC and achievable
- Save user preferences for future sessions
- Update todos as you complete each task"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [51]:
# Step 4: Test with a user creating their 30-day challenge
print("\n" + "="*60)
print("🎯 TEST 1: Creating 30-Day Wellness Challenge")
print("="*60 + "\n")

TODO_STORE.clear()

result1 = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_sarah. I want to start a 30-Day Wellness Challenge!

About me:
- Fitness level: Beginner (haven't exercised regularly in 2 years)
- Diet: Vegetarian, trying to eat healthier
- Time available: 30-45 minutes per day (mornings work best)
- Goals: Lose weight, have more energy, sleep better, reduce work stress
- Constraints: No gym access (home workouts only), sometimes travel for work

Please create a comprehensive 30-day challenge for me with all the plans saved as separate files!"""
    }]
})

print("🤖 Coach Response:")
print(result1["messages"][-1].content)

print("\n" + "="*60)
print("📋 Todo Status After Challenge Creation")
print("="*60)
print(list_todos.invoke({}))



🎯 TEST 1: Creating 30-Day Wellness Challenge

🤖 Coach Response:
Sarah, this is exciting—your personalized 30-Day Wellness Challenge is ready! I saved all plans as separate files and organized everything for easy use. Here’s what you’ve got and where to find it:

Your files
- 30-Day Integrated Plan: /user_sarah_30day_challenge.md
- Exercise Plan (detailed): /user_sarah_exercise_plan.md
- Nutrition Guide + Templates + Recipes: /user_sarah_nutrition_guide.md
- Mindfulness/Sleep Routine: /user_sarah_mindfulness_routine.md
- Weekly Checklist (simple tracking): /user_sarah_weekly_checklist.md

Highlights tailored to you
- Time: 30–45 minutes/day, mornings
- Fitness: Beginner-friendly, low-impact, with knee-friendly mods
- Equipment: Home-based (mat, bands, light dumbbells optional). Travel-ready swaps included
- Nutrition: Vegetarian, simple 10–30 min meals, batch-cook options, no-cook travel days
- Stress & Sleep: 2–5 min AM breath stack, 1–3 min work micro-breaks, 10–15 min PM wind-down
-

In [52]:
# Step 5: Simulate a daily check-in and adaptation
print("\n" + "="*60)
print("📊 FINAL SYSTEM REVIEW")
print("="*60)


print("\n📝 CHECK-IN HISTORY:")
if "user_sarah" in CHECKIN_LOG:
    print(json.dumps(CHECKIN_LOG["user_sarah"], indent=2))

print("\n📁 GENERATED FILES:")
for f in sorted(workspace_path.iterdir()):
    if f.is_file():
        size = f.stat().st_size
        print(f"  📄 {f.name} ({size} bytes)")
    elif f.is_dir():
        print(f"  📁 {f.name}/")

# Show sample file content
files = list(workspace_path.glob("*30day_challenge.md"))
if files:
    print(f"\n📖 SAMPLE: {files[0].name}")
    print("="*60)
    content = files[0].read_text()
    print(content[:1500] + "..." if len(content) > 1500 else content)


📊 FINAL SYSTEM REVIEW

📝 CHECK-IN HISTORY:

📁 GENERATED FILES:
  📁 dietary_guides/
  📁 docs/
  📄 equipment_alternatives.txt (651 bytes)
  📄 form_cues_reference.txt (554 bytes)
  📄 HealthWellnessGuide.txt (16624 bytes)
  📄 HealthWellnessGuide_Summary.txt (1525 bytes)
  📄 nutrition_plan_alex.txt (4259 bytes)
  📁 research/
  📄 sleep_improvement_plan.txt (3470 bytes)
  📄 user_alex_exercise_plan_2w.txt (5748 bytes)
  📄 user_alex_nutrition_plan_2w.txt (8947 bytes)
  📄 user_alex_stress_sleep_plan_2w.txt (4783 bytes)
  📄 user_sarah_30day_challenge.md (5622 bytes)
  📄 user_sarah_exercise_plan.md (7404 bytes)
  📄 user_sarah_mindfulness_routine.md (2378 bytes)
  📄 user_sarah_nutrition_guide.md (5827 bytes)
  📄 user_sarah_stress_management_sleep_routine.md (3605 bytes)
  📄 user_sarah_weekly_checklist.md (1561 bytes)
  📄 weekly_progression_notes.txt (394 bytes)
  📄 workout_plan_week_1.txt (3092 bytes)
  📄 workout_plan_week_2.txt (1973 bytes)
  📄 workout_plan_week_3.txt (1892 bytes)
  📄 workout_pla

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)